In [456]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords

In [457]:
df = pd.read_csv('data.csv')

In [458]:
df.head()

,Date,Platform,Text,Person,Truthfulness Category
0,"December 9, 2022",reel,The Federal Reserve “has suspended the availab...,Facebook posts,false
1,"December 3, 2022",social post,“The revelation of massive & widespread fraud ...,Donald Trump,pants-fire
2,"November 28, 2022",post,The 2021 infrastructure bill requires all new ...,Instagram posts,barely-true
3,"November 27, 2022",cnn,“60% of counties in this country are refusing ...,Chris Murphy,half-true
4,"November 22, 2022",public statement,“A lot of the programs (that were) started dur...,Robin Vos,barely-true


In [459]:
df.shape

(5430, 5)

In [460]:
month = []
day = []
year = []
for i in range(df['Date'].size):
    s = (df['Date'].iloc[i]).split()
    day.append(s[1])
    year.append(s[2])
    month.append(s[0])

In [461]:
mdf = pd.DataFrame(df)
ddf = pd.DataFrame(df)
ydf = pd.DataFrame(df)

In [462]:
mdf['Month'] = month
ddf['Day'] = day
ydf['Year'] = year

In [463]:
df = df.drop(columns=['Date'], axis=1)
lb = LabelEncoder()
df['Month'] = lb.fit_transform(df['Month'])
df['Day'] = lb.fit_transform(df['Day'])
df['Year'] = lb.fit_transform(df['Year'])
df['Platform'] = lb.fit_transform(df['Platform'])
df['Person'] = lb.fit_transform(df['Person'])

In [464]:
df['Truthfulness Category'] = df['Truthfulness Category'].replace(['mostly-true'], 'true')
df['Truthfulness Category'] = df['Truthfulness Category'].replace(['pants-fire'], 'false')
df['Truthfulness Category'] = df['Truthfulness Category'].replace(['barely-true'], 'false')
df['Truthfulness Category'] = df['Truthfulness Category'].replace(['half-true'], 'false')

In [465]:
df['Truthfulness Category'].value_counts()

false        3915
true         1441
full-flop      53
half-flip      15
no-flip         6
Name: Truthfulness Category, dtype: int64

In [466]:
df.drop(df[df['Truthfulness Category'] == 'full-flop'].index, inplace = True)
df.drop(df[df['Truthfulness Category'] == 'half-flip'].index, inplace = True)
df.drop(df[df['Truthfulness Category'] == 'no-flip'].index, inplace = True)
df['Truthfulness Category'].value_counts()

false    3915
true     1441
Name: Truthfulness Category, dtype: int64

In [467]:
df['Truthfulness Category'] = lb.fit_transform(df['Truthfulness Category'])

In [468]:
#  Preprocess train dataset
# remove special characters from text column
df.Text = df.Text.str.replace('[#,@,&]', '')
# Remove digits
df.Text = df.Text.str.replace('\d*','')
#Remove www
df.Text = df.Text.str.replace('w{3}','')
# remove urls
df.Text = df.Text.str.replace("http\S+", "")
# remove multiple spaces with single space
df.Text = df.Text.str.replace('\s+', ' ')
#remove all single characters
df.Text = df.Text.str.replace(r'\s+[a-zA-Z]\s+', '')
stop_words = stopwords.words('english')
# Remove english stopwords
df['Text'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

C:\Users\Janmejay Mohanty\AppData\Local\Temp\ipykernel_15540\97837966.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Text = df.Text.str.replace('[#,@,&]', '')
C:\Users\Janmejay Mohanty\AppData\Local\Temp\ipykernel_15540\97837966.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Text = df.Text.str.replace('\d*','')
C:\Users\Janmejay Mohanty\AppData\Local\Temp\ipykernel_15540\97837966.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Text = df.Text.str.replace('w{3}','')
C:\Users\Janmejay Mohanty\AppData\Local\Temp\ipykernel_15540\97837966.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Text = df.Text.str.replace("http\S+", "")
C:\Users\Janmejay Mohanty\AppData\Local\Temp\ipykernel_15540\97837966.py:11: FutureWarning: The default value of regex will change from True 

In [469]:
X = df.drop(columns=['Truthfulness Category'], axis=1)
Y = df['Truthfulness Category']

In [470]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3,random_state=42) 
print(X_train.shape ,X_test.shape, y_train.shape, y_test.shape)

(3749, 6) (1607, 6) (3749,) (1607,)


In [471]:
model = LogisticRegression()
model.fit(X_train, y_train)
p = model.predict(X_test)
accuracy_score(y_test, p)

ValueError: could not convert string to float: '"The McCain campaign roughly position Vice President Gore running President Bush one week election ."'

In [ ]:
counter = CountVectorizer()
counter.fit_transform(X_train)


#count the number of times each term appears in a document and transform each doc into a count vector
counts_train = counter.transform(X_train)#transform the training data
counts_test = counter.transform(X_test)#transform the testing data

In [ ]:
clf3 =  svm.SVC()
clf3.fit(counts_train, y_train)

# evaluate classifier's accuracy
accuracy = clf3.score(counts_test, y_test)
print("Accuracy:", accuracy)

ValueError: Found input variables with inconsistent numbers of samples: [6, 3749]

In [ ]:
clf = svm.SVC(C=1.4, kernel='linear', gamma='auto')

#train all classifier on the same datasets
clf.fit(counts_train,y_train)

#use hard voting to predict (majority voting)
pred=clf.predict(counts_test)

#print accuracy
print(accuracy_score(pred,y_test)*100)

ValueError: Found input variables with inconsistent numbers of samples: [6, 3749]